In [1]:
:load KrapshDisplays KrapshDagDisplay

In [2]:
:extension DeriveGeneric
:extension FlexibleContexts
:extension OverloadedStrings
:extension GeneralizedNewtypeDeriving
:extension FlexibleInstances
:extension MultiParamTypeClasses

In [3]:
import Spark.Core.Dataset
import Spark.Core.Context
import Spark.Core.Column
import Spark.Core.ColumnFunctions
import Spark.Core.Functions
import Spark.Core.Row
import Spark.Core.Types
import Spark.Core.Try

import qualified Data.Vector as V
import qualified Data.Text as T
import GHC.Generics

In [4]:
myScaler :: Column ref Double -> Column ref Double
myScaler col =
  let cnt = asDouble (countCol col) @@ "count"
      m = sumCol col / cnt @@ "mean"
      centered = col .- m
      stdDev = sumCol (centered * centered) / cnt @@ "std_dev"
  in centered ./ stdDev


In [5]:
:t myScaler

myScaler :: forall ref. Column ref Double -> Column ref Double

In [6]:
let ds = dataset [-1, 1] :: Dataset Double
let c = myScaler (asCol ds)

In [7]:
let ds2 = pack1 c

In [8]:
showGraph ds2

'>

In [9]:
let res = collect (asCol ds2)

In [10]:
conf = defaultConf {
        confEndPoint = "http://10.0.2.2",
        confRequestedSessionName = "col_ops6" }

createSparkSessionDef conf

[Debug] Creating spark session at url: http://10.0.2.2:8081/sessions/col_ops6 @(<unknown>:<unknown> <unknown>:0:0)

In [11]:
exec1Def res

[Debug] executeCommand1: computing observable collect_659486@org.spark.Collect![double] @(<unknown>:<unknown> <unknown>:0:0)
[Info] Sending computations at url: http://10.0.2.2:8081/computations/col_ops6/0/create @(<unknown>:<unknown> <unknown>:0:0)
[Debug] executeCommand1: Tracked nodes are [(d0dee..,NPath(sum_d0deee),double,sum_d0deee),(6bec8..,NPath(count_6bec8b),int,count_6bec8b),(962b2..,NPath(count),double,count),(4007b..,NPath(mean),double,mean),(53b70..,NPath(localpack_53b709),{_1:double _2:double},localpack_53b709),(a130c..,NPath(sum_a130cf),double,sum_a130cf),(33298..,NPath(std_dev),double,std_dev),(beff5..,NPath(localpack_beff5a),{_1:double _2:double},localpack_beff5a),(65948..,NPath(collect_659486),[double],collect_659486)] @(<unknown>:<unknown> <unknown>:0:0)
[Info] _computationMultiStatus: /sum_d0deee running @(<unknown>:<unknown> <unknown>:0:0)
[Info] _computationMultiStatus: /count_6bec8b running @(<unknown>:<unknown> <unknown>:0:0)
[Info] _computationMultiStatus: /sum_

In [14]:
import Spark.Core.StructuresInternal(ComputationID(..))
import IHaskell.Display
stats <- computationStatsDef (ComputationID "0")

[Debug] computationStats: stats for ComputationID {unComputationID = "0"} @(<unknown>:<unknown> <unknown>:0:0)
[Debug] Sending computations stats request at url: http://10.0.2.2:8081/computations_status/col_ops6/0/ @(<unknown>:<unknown> <unknown>:0:0)

In [15]:
let ns = statsToExportNodes stats
let c = T.unpack (tfIFrame ns)
Display [html c]

'>